## הקדמה

בשיעור זה נעסוק ב:
- מהו קריאה לפונקציה ובאילו מקרים משתמשים בה
- איך ליצור קריאה לפונקציה באמצעות Azure OpenAI
- איך לשלב קריאה לפונקציה בתוך אפליקציה

## מטרות למידה

לאחר סיום השיעור תדעו ותבינו:

- את המטרה של שימוש בקריאה לפונקציה
- איך להגדיר קריאה לפונקציה באמצעות Azure Open AI Service
- איך לעצב קריאות פונקציה יעילות עבור המקרה השימושי של האפליקציה שלכם


## הבנת קריאות לפונקציות

בשיעור הזה, נבנה פיצ'ר לסטארטאפ החינוכי שלנו שמאפשר למשתמשים להשתמש בצ'אטבוט כדי למצוא קורסים טכנולוגיים. נמליץ על קורסים שמתאימים לרמת הידע שלהם, לתפקיד הנוכחי ולטכנולוגיה שמעניינת אותם.

כדי להשלים את זה נשתמש בשילוב של:
 - `Azure Open AI` כדי ליצור חוויית צ'אט למשתמש
 - `Microsoft Learn Catalog API` כדי לעזור למשתמשים למצוא קורסים לפי הבקשה שלהם
 - `Function Calling` כדי לקחת את השאילתה של המשתמש ולשלוח אותה לפונקציה שמבצעת את הבקשה ל-API.

כדי להתחיל, בואו נבין למה בכלל נרצה להשתמש בקריאות לפונקציות:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # קבלת תגובה חדשה מ-GPT שבה הוא יכול לראות את התגובה של הפונקציה


print(second_response.choices[0].message)


### למה להשתמש ב-Function Calling

אם השלמת שיעור אחר בקורס הזה, כנראה שאתה כבר מבין את הכוח של שימוש במודלים גדולים לשפה (LLMs). סביר להניח שגם הבחנת בכמה מהמגבלות שלהם.

Function Calling היא תכונה בשירות Azure Open AI שנועדה להתגבר על המגבלות הבאות:
1) פורמט תגובה עקבי
2) היכולת להשתמש בנתונים ממקורות אחרים של אפליקציה בתוך שיחה

לפני שהייתה אפשרות ל-Function Calling, התגובות ממודל השפה היו לא מובנות ולא עקביות. מפתחים היו צריכים לכתוב קוד אימות מסובך כדי לוודא שהם יכולים להתמודד עם כל וריאציה של תגובה.

משתמשים לא יכלו לקבל תשובות כמו "מה מזג האוויר הנוכחי בסטוקהולם?". הסיבה לכך היא שהמודלים היו מוגבלים לזמן שבו הנתונים שלהם אומנו.

בואו נסתכל על הדוגמה למטה שממחישה את הבעיה הזו:

נניח שאנחנו רוצים ליצור מסד נתונים של נתוני תלמידים כדי שנוכל להמליץ להם על הקורס המתאים. למטה יש לנו שתי תיאורים של תלמידים שמכילים נתונים דומים מאוד.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

אנחנו רוצים לשלוח את זה למודל שפה גדול (LLM) כדי לנתח את הנתונים. מאוחר יותר נוכל להשתמש בזה באפליקציה שלנו כדי לשלוח את זה ל-API או לשמור את זה במסד נתונים.

בואו ניצור שני פרומפטים זהים שבהם ננחה את ה-LLM אילו נתונים מעניינים אותנו:


אנחנו רוצים לשלוח את זה למודל שפה גדול כדי לנתח את החלקים שחשובים למוצר שלנו. כך נוכל ליצור שני הנחיות זהות כדי להנחות את המודל:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


לאחר יצירת שני ההנחיות הללו, נשלח אותן ל-LLM באמצעות ‎`openai.ChatCompletion`‎. אנו שומרים את ההנחיה במשתנה ‎`messages`‎ ומקצים את התפקיד ל-‎`user`‎. זאת כדי לדמות הודעה ממשתמש הנכתבת לצ'אטבוט.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

עכשיו אנחנו יכולים לשלוח את שתי הבקשות ל-LLM ולבחון את התגובה שנקבל.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

למרות שההנחיות זהות והתיאורים דומים, אנחנו יכולים לקבל פורמטים שונים של המאפיין `Grades`.

אם תריץ את התא למעלה כמה פעמים, הפורמט יכול להיות `3.7` או `3.7 GPA`.

הסיבה לכך היא שהמודל הלשוני מקבל נתונים לא מובנים בצורה של הנחיה כתובה וגם מחזיר נתונים לא מובנים. אנחנו צריכים פורמט מובנה כדי שנדע למה לצפות כשאנחנו שומרים או משתמשים בנתונים האלה.

באמצעות קריאה פונקציונלית, אפשר לוודא שמקבלים נתונים מובנים בחזרה. כשמשתמשים בקריאה פונקציונלית, המודל הלשוני לא באמת מפעיל או מריץ פונקציות. במקום זאת, אנחנו יוצרים מבנה שהמודל צריך לעקוב אחריו בתשובות שלו. לאחר מכן, אנחנו משתמשים בתשובות המובנות כדי לדעת איזו פונקציה להפעיל באפליקציות שלנו.


![דיאגרמת זרימת קריאות פונקציה](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.he.png)


### שימושים עיקריים לקריאות לפונקציות

**שימוש בכלים חיצוניים**  
צ'אטבוטים מצוינים במתן תשובות לשאלות של משתמשים. בעזרת קריאות לפונקציות, הצ'אטבוטים יכולים להשתמש בהודעות מהמשתמשים כדי לבצע משימות מסוימות. לדוגמה, סטודנט יכול לבקש מהצ'אטבוט "שלח מייל למרצה שלי וכתוב שאני צריך עזרה נוספת בנושא הזה". זה יכול להפעיל קריאה לפונקציה `send_email(to: string, body: string)`

**יצירת שאילתות ל-API או למסדי נתונים**  
משתמשים יכולים למצוא מידע בעזרת שפה טבעית שמומרת לשאילתה או בקשה ל-API בפורמט מתאים. לדוגמה, מורה יכול לשאול "מי התלמידים שסיימו את המשימה האחרונה", מה שיכול להפעיל פונקציה בשם `get_completed(student_name: string, assignment: int, current_status: string)`

**יצירת נתונים מובנים**  
משתמשים יכולים לקחת קטע טקסט או קובץ CSV ולהיעזר ב-LLM כדי להוציא ממנו מידע חשוב. לדוגמה, סטודנט יכול להמיר ערך מוויקיפדיה על הסכמי שלום לכרטיסיות לימוד מבוססות בינה מלאכותית. זה יכול להתבצע בעזרת פונקציה בשם `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. יצירת קריאת הפונקציה הראשונה שלך

תהליך יצירת קריאת פונקציה כולל שלושה שלבים עיקריים:
1. קריאה ל-Chat Completions API עם רשימת הפונקציות שלך והודעה מהמשתמש
2. קריאת התגובה של המודל כדי לבצע פעולה, כלומר להריץ פונקציה או קריאה ל-API
3. ביצוע קריאה נוספת ל-Chat Completions API עם התגובה מהפונקציה שלך, כדי להשתמש במידע הזה ליצירת תגובה למשתמש.


![זרימת קריאה לפונקציה](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.he.png)


### מרכיבי קריאה לפונקציה

#### קלט מהמשתמש

השלב הראשון הוא ליצור הודעה מהמשתמש. אפשר להקצות אותה באופן דינמי על ידי קבלת הערך מקלט טקסט, או להכניס ערך כאן. אם זו הפעם הראשונה שלך לעבוד עם Chat Completions API, צריך להגדיר את ה-`role` ואת ה-`content` של ההודעה.

ה-`role` יכול להיות או `system` (הגדרת כללים), `assistant` (המודל) או `user` (המשתמש הסופי). עבור קריאה לפונקציה, נגדיר זאת כ-`user` ונשתמש בשאלה לדוגמה.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### יצירת פונקציות.

כעת נגדיר פונקציה ואת הפרמטרים שלה. כאן נשתמש בפונקציה אחת בלבד בשם `search_courses`, אבל אפשר ליצור כמה פונקציות שתרצו.

**חשוב**: פונקציות נכללות בהודעת המערכת ל-LLM והן יחשבו כחלק ממספר הטוקנים הזמינים לכם.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**הגדרות**

`name` - השם של הפונקציה שברצוננו שתקרא.

`description` - זהו תיאור של איך הפונקציה פועלת. כאן חשוב להיות מדויקים וברורים.

`parameters` - רשימה של ערכים והפורמט שתרצה שהמודל יחזיר בתשובתו.

`type` - סוג הנתונים שבו הערכים יאוחסנו.

`properties` - רשימה של הערכים הספציפיים שהמודל ישתמש בהם בתשובתו.

`name` - שם המאפיין שהמודל ישתמש בו בתשובה המפורמטת שלו.

`type` - סוג הנתונים של מאפיין זה.

`description` - תיאור של המאפיין הספציפי.

**אופציונלי**

`required` - מאפיין שחובה לספק כדי שהקריאה לפונקציה תושלם.


### ביצוע קריאה לפונקציה
לאחר שהגדרנו פונקציה, כעת עלינו לכלול אותה בקריאה ל-Chat Completion API. אנחנו עושים זאת על ידי הוספת `functions` לבקשה. במקרה הזה `functions=functions`.

יש גם אפשרות להגדיר את `function_call` ל-`auto`. המשמעות היא שניתן למודל השפה להחליט איזו פונקציה יש לקרוא על סמך ההודעה של המשתמש, במקום שנקבע זאת בעצמנו.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

עכשיו בואו נסתכל על התגובה ונראה איך היא מעוצבת:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

אתם יכולים לראות ששם הפונקציה נקרא ומההודעה של המשתמש, המודל הצליח למצוא את הנתונים שמתאימים לארגומנטים של הפונקציה.


## 3. שילוב קריאות לפונקציות בתוך אפליקציה

אחרי שבדקנו את התגובה המעוצבת מה-LLM, עכשיו אפשר לשלב אותה בתוך אפליקציה.

### ניהול זרימת העבודה

כדי לשלב את זה באפליקציה שלנו, נבצע את הצעדים הבאים:

ראשית, נבצע קריאה לשירותי Open AI ונשמור את ההודעה במשתנה בשם `response_message`.


In [ ]:
response_message = response.choices[0].message

עכשיו נגדיר את הפונקציה שתקרא ל-API של Microsoft Learn כדי לקבל רשימת קורסים:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



כמנהג מומלץ, נבדוק אם המודל רוצה לקרוא לפונקציה. לאחר מכן, ניצור אחת מהפונקציות הזמינות ונתאים אותה לפונקציה שנקראה.
לאחר מכן ניקח את הארגומנטים של הפונקציה ונמפה אותם לארגומנטים מה-LLM.

לבסוף, נוסיף את הודעת קריאת הפונקציה ואת הערכים שהוחזרו מההודעה `search_courses`. זה נותן ל-LLM את כל המידע שהוא צריך
כדי להגיב למשתמש בשפה טבעית.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## אתגר קוד

עבודה מצוינת! כדי להמשיך ללמוד על Azure Open AI Function Calling, אתם יכולים לבנות: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - להוסיף עוד פרמטרים לפונקציה שיכולים לעזור ללומדים למצוא קורסים נוספים. תוכלו למצוא את כל הפרמטרים הזמינים של ה-API כאן:  
 - ליצור קריאה נוספת לפונקציה שלוקחת מידע נוסף מהלומד, כמו שפת האם שלו  
 - להוסיף טיפול בשגיאות כאשר קריאת הפונקציה ו/או קריאת ה-API לא מחזירה קורסים מתאימים



---

Certainly! Here is your text translated into Hebrew:

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עשויים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפת המקור ייחשב כמקור הסמכותי. למידע קריטי, מומלץ לפנות לתרגום מקצועי על ידי אדם. איננו אחראים לכל אי-הבנה או פרשנות שגויה הנובעת מהשימוש בתרגום זה.
